In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer

model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [6]:
text = "Quote: Imagination is more"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Imagination is more important than knowledge. Knowledge is limited. Imagination encircles the world.

- Albert Einstein

The


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
from google.colab import files
# Upload your CSV file
print("Please upload your CSV file when prompted...")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]  # Get the name of the uploaded file

Please upload your CSV file when prompted...


Saving Gemma 2B Training Data - Sheet1.csv to Gemma 2B Training Data - Sheet1 (1).csv


In [10]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [11]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [50]:
import pandas as pd
from datasets import Dataset


# Define formatting function once to be used in both places
def formatting_func(example):
    # Using input_text and output as our column names
    text = f"Input: {example['text_input']}\nOutput: {example['output']}<eos>"
    return [text]

def prepare_dataset(file_name, tokenizer, max_length=512):
    print("Reading CSV file...")
    df = pd.read_csv(file_name)

    # Create dataset from pandas
    dataset = Dataset.from_pandas(df)

    print("Tokenizing dataset...")
    # Don't remove the original columns
    tokenized_dataset = dataset.map(
        lambda examples: tokenizer(
            [f"Input: {input_text}\nOutput: {output}<eos>"
             for input_text, output in zip(examples['text_input'], examples['output'])],
            truncation=True,
            max_length=max_length
        ),
        batched=True
    )

    return tokenized_dataset

In [51]:
# Create dataset
data = prepare_dataset(file_name, tokenizer)

Reading CSV file...
Tokenizing dataset...


Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [53]:
import transformers
from trl import SFTTrainer

# Create trainer using the same formatting_func
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        report_to="none"  # Removes WANDB warning
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
    packing=False,
    max_seq_length=512  # Removes max_seq_length warning
)

# Start training
trainer.train()

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Step,Training Loss
1,0.709400
2,0.709400
3,0.697300
4,0.669700
5,0.644100
6,0.621000
7,0.601300
8,0.585200
9,0.572900
10,0.564100


TrainOutput(global_step=10, training_loss=0.6374627768993377, metrics={'train_runtime': 2.4893, 'train_samples_per_second': 16.069, 'train_steps_per_second': 4.017, 'total_flos': 61184724172800.0, 'train_loss': 0.6374627768993377, 'epoch': 10.0})

In [66]:
trainer.save_model("/content/drive/MyDrive/gemma_ft_saved")

In [63]:
text = "Input: چطور میتوان در پوهنتون های امریکا ثبت نام کرد؟ Output: "
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)

In [64]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Input: چطور میتوان در پوهنتون های امریکا ثبت نام کرد؟ Output: 1. Registering in American universities 2. How to register in American universities 3. How
